# 4. Debugging

It is often observed in genome scale models that adding thermodynamic constraints make the model infeasible. This might be due to variety of reasons, like ill-formatted  covariance between groups/components, lumping of reactions etc. It is always not easy to find the exact cause of the problem. But, if you are using Gurobi/Cplex, you can find the constraints that render the model infeasible. 

Please note that these methods give you random set of constraints that represent infeasibility. Also, a model can contain multiple infeasible sets. While modifying/removing these constraints, it is advisable to look for biological meaning.

Let us demonstrate the usage by breaking the model. 


In [1]:
import sys
sys.path.insert(0, "../../examples")
from paper_data_example import build_core_model

Fragments already downloaded
Fragments already downloaded
Fragments already downloaded


In [2]:
tfa_model = build_core_model()

Now lets change the concentration **atp** and find the objective value. Then, we set the lower bound of objective variable higher than the solution to break the model.

In [3]:
atp = tfa_model.metabolites.get_by_id('atp_c')
atp.concentration_min = 2e-3
atp.concentration_max = 5e-2
solution = tfa_model.optimize()
print(solution)



Selected objective sense:  MINIMIZE
Selected objective  name:  d34bcb4f-5dba-11eb-bac2-c1c11d7a5b17
Selected RHS        name:  rhs
Selected bound      name:  bnd
<Solution -0.374 at 0x7f401e1fb550>


Now lets adjust the biomass lower bound to 0.5 and find out the IIS set of reactions using `Gurobi`

In [4]:
biomass = tfa_model.reactions.get_by_id('BIOMASS_Ecoli_core_w_GAM')
biomass.lower_bound = 0.5
solution = tfa_model.optimize()
tfa_model.solver.problem.computeIIS() # model.solver.problem represents the solver interface model
for cons in tfa_model.solver.problem.getConstrs(): 
    if cons.IISConstr: 
        print(cons) 

AttributeError: 'Cplex' object has no attribute 'computeIIS'

This returned us a list of constraints that can be modified to make the model status optimal. One has to choose and modify the list of constraints to manipulate based on biological meaning.